# Yolov5 Object Detection with OpenVINO™ Toolkit™

The YOLOv5 algorithm is making big waves in the computer vision and machine learning communities.
It is real-time object detection algorithm that performs image recognition tasks by taking an image as input 
and then predicting bounding boxes and class probabilities for each object in the image.

YOLO stands for “You Only Look Once”, it is a popular family of real-time object detection algorithms. 
The original YOLO object detector was first released in 2016. Since then, different versions and variants of YOLO have been proposed, each providing a significant increase in performance and efficiency.
YOLOv7 is next stage of evalution of YOLO models family which provides a greatly improved real-time object detection accuracy without increasing the inference costs.
More details about its realization can be found in original model [paper](https://arxiv.org/abs/2207.02696) and [repository](https://github.com/WongKinYiu/yolov7)

Real-time object detection is often used as a key component in computer vision systems. 
Applications that use real-time object detection models include video analytics, robotics, autonomous vehicles, multi-object tracking and object counting, medical image analysis, and many others.


This tutorial demonstrates step-by-step instructions on how to run and optimize PyTorch\* Yolo V7 with OpenVINO.

The tutorial consists of the following steps:
- Prepare PyTorch model
- Download and prepare dataset
- Validate original model
- Convert PyTorch model to ONNX
- Convert ONNX model to OpenVINO IR
- Validate converted model

## Install OpenVINO in Amazon SageMaker Studio

Instructions to Run the OpenVINO Notebooks in [Amazon SageMaker Studio](https://github.com/openvinotoolkit/openvino_notebooks/wiki/SageMaker).

## Install OpenVINO in local environment

Instructions to install OpenVINO on [local machine](https://github.com/openvinotoolkit/openvino_notebooks#-installation-guide).

## Proceed to next steps once an installation path has been chosen and configured

## Get Pytorch model

Generally, PyTorch models represent an instance of the [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) class, initialized by a state dictionary with model weights.
We will use the YOLOv7 tiny model pretrained on COCO dataset, which available in this [repo](https://github.com/WongKinYiu/yolov7).
Typical steps to obtain pretrained model:
1. Create instance of model class
2. Load checkpoint state dict, which contains pretrained model weights
3. Turn model to evaluation for switching some operations to inference mode

In this case, the model creators provide a tool that enables converting the YOLOv7 model to ONNX, so we don't need to do these steps manually.

## Prerequisites

In [ ]:
import sys
from pathlib import Path
sys.path.append("../utils")
from notebook_utils import download_file
from IPython.display import Markdown, display

In [ ]:
# Clone YOLOv5 repo
if not Path('yolov5').exists():
    !git clone https://github.com/ultralytics/yolov5
%cd yolov5

In [ ]:
print("Convert PyTorch model to OpenVINO Model:")
command_export = f'{"python export.py --weights yolov5s/yolov5s.pt --imgsz 640 --batch-size 1 --include openvino"}'
display(Markdown(f"`{command_export}`"))
! $command_export

Download the data and trained model to the directory.

More details can be found in [Yolov5 Object Detection with OpenVINO™ Toolkit™](https://github.com/AnishaUdayakumar/YoloV5-Football-Tracker/blob/main/README.md) notebook repo.

In [ ]:
!git clone https://github.com/AnishaUdayakumar/YoloV5-Football-Tracker.git

In [ ]:
!python detect.py --weights YoloV5-Football-Tracker/football-players-tracking.pt --img-size 640 --source YoloV5-Football-Tracker/Capture_PNG.rf.f6303bf93745f5fc96343b9929f0bb37.jpg  --device cpu

Replace the "Results" path from above to visualize the prediction result

In [ ]:
from PIL import Image
# visualize prediction result
Image.open('runs/detect/exp3/Capture_PNG.rf.f6303bf93745f5fc96343b9929f0bb37.jpg')

## Export to ONNX

To export and ONNX format of the model, we will use `export.py` script. Let's check it's arguments.
Including whole post-processing to model can help to achieve more perfomant resuls, but in the same time it makes model less flexible and does not guarantee full accuracy reproducibility. 
It is the reason why we will add only `--grid` parameter to preserve original pytorch model result format.

In [ ]:
!python export.py --weights YoloV5-Football-Tracker/football-players-tracking.pt --include openvino

## Check model inference

`detect.py` script run pytorch model inference and save image as result,

In [ ]:
!python detect.py --weights YoloV5-Football-Tracker/football-players-tracking_openvino_model --img-size 640 --source YoloV5-Football-Tracker/Capture_PNG.rf.f6303bf93745f5fc96343b9929f0bb37.jpg  --device cpu

Replace the "Results" path from above to visualize the prediction result

In [ ]:
from PIL import Image
# visualize prediction result
Image.open('runs\detect\exp4\Capture_PNG.rf.f6303bf93745f5fc96343b9929f0bb37.jpg')

In [ ]:
# Inference FP32 model (OpenVINO IR)
!benchmark_app -m YoloV5-Football-Tracker/football-players-tracking.onnx -d CPU -api async

In [ ]:
# Inference INT8 model (OpenVINO IR)
!benchmark_app -m YoloV5-Football-Tracker/football-players-tracking_openvino_model/football-players-tracking.xml -d CPU -api async